In [1]:
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/

In [2]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.preprocessing import StandardScaler, MinMaxScaler



In [3]:
file = '../data/Hz31.txt'
data = pd.read_csv(file, names=['z', 'hz', 'err'], sep = " " )
# besthzorder = np.array([25,  3,  9,  7, 19,  2, 17, 16,  6, 28, 12, 20, 21, 26, 27, 11,  1,
#        10, 30, 13, 29,  0,  8, 18, 24,  5, 23,  4, 22, 15, 14])
# data = data.values[a]

# file = 'data/fs8Diagram.txt'
# data = pd.read_csv(file, names=['z', '$f\sigma_8$', 'err', '$\Omega_ref$'], sep = "\t", skiprows=3)
# data = data[['z', '$f\sigma_8$', 'err']]
# randomize = np.random.permutation(len(data.values))
# a = np.array([21,1,3,10,12,17,8,4,7,16, 2,11,19,13,18,5,9,20,15,14,6,0])
even = [x for x in range(len(data.values)) if x%2 ==0]
odd = [x for x in range(len(data.values)) if x%2 ==1]
even.extend(odd)
order = np.array(even)

data = data.values[order]


X = data[:,0]
y = data[:,1:]

scalerx = StandardScaler()
# scalerz = MinMaxScaler(feature_range=(-1,1))
# fit scaler on data
scalerx.fit(X.reshape(-1,1))
# apply transform
X = scalerx.transform(X.reshape(-1,1))


# scalery = MinMaxScaler(feature_range=(0,1))
# scalery = StandardScaler()
# scalery.fit(y)
# y = scalery.transform(y)

split = 0.8
ntrain = int(split * len(X))
indx = [ntrain]
X_train, X_test = np.split(X, indx)
y_train, y_test = np.split(y, indx)


In [4]:
# loss -> val_loss
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min',
                                   min_delta=0.01,
                                   patience=50,
                                   restore_best_weights=True)
                                   ]

n_cols = 1

In [5]:
HP_NUM_UNITS1 = hp.HParam('num_units1', hp.Discrete([50, 100, 150, 200]))
HP_NUM_UNITS2 = hp.HParam('num_units2', hp.Discrete([50, 100, 150, 200]))
HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([50, 100, 150, 200]))
HP_NUM_UNITS4 = hp.HParam('num_units4', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS4 = hp.HParam('num_units4', hp.Discrete([2, 5, 10]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.2))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'Adadelta']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))
HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([1, 4, 8]))

In [6]:
# sess = tf.Session()
# writer = tf.summary.FileWriter("/tmp/tfvgg", sess.graph)
# init = tf.initialize_all_variables()
# sess.run(init)
# with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
#     hp.hparams_config(
#         hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_NUM_UNITS4,
#                  HP_OPTIMIZER, HP_BATCHSIZE],
#         metrics=[hp.Metric('loss', display_name="Loss")])

In [7]:
# METRIC_ACCURACY = 'accuracy'
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
# with tf.summary.FileWriter('logs/hparam_tuning', sess.graph):
#     init = tf.initialize_all_variables()
#     sess.run(init)
    hp.hparams_config(
        hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_NUM_UNITS4, HP_BATCHSIZE],
        metrics=[hp.Metric('loss', display_name="Loss")])

In [8]:
def hzmodel(input_z, hparams):
    #encoder
    efirst = Dense(hparams[HP_NUM_UNITS1], activation='relu', input_shape=(1,))(input_z)
    x = Dense(hparams[HP_NUM_UNITS2], activation='relu')(efirst)
    x = Dense(hparams[HP_NUM_UNITS3], activation='relu')(x)
    x = Dense(hparams[HP_NUM_UNITS4], activation='relu')(x)
#     ehidden2 = Dense(hparams[HP_NUM_UNITS3], activation='relu')(ehidden1)
    elast = Dense(2, activation='linear')(x)
        
    return elast

In [9]:
epochs = 200
input_z = tf.keras.layers.Input(shape = (1,))

In [10]:
def train_test_model(hparams):
    model = Model(input_z, hzmodel(input_z, hparams))  
                              
    model.compile(
#             optimizer=hparams[HP_OPTIMIZER],
            optimizer='adam',
            loss='mean_squared_error',
            metrics=['mean_squared_error']
          )
    # Run with 1 epoch to speed things up for demo purposes
    # model.fit(x_train, y_train, epochs=1, callbacks=callbacks) 
    # _, loss = model.evaluate(x_test, y_test)
    model.fit(X_train, y_train, epochs=epochs, 
              batch_size=hparams[HP_BATCHSIZE], callbacks=callbacks, verbose=0) 
    _, loss = model.evaluate(X_test, y_test)


    return loss
    

In [11]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
#     with tf.summary.FileWriter(run_dir):
        hp.hparams(hparams)  # record the values used in this trial
        loss = train_test_model(hparams)
        tf.summary.scalar("loss", loss, step=1)

In [13]:
session_num = 0

for batch_size in HP_BATCHSIZE.domain.values:
    for num_units1 in HP_NUM_UNITS1.domain.values:
        for num_units2 in HP_NUM_UNITS2.domain.values:
            for num_units3 in HP_NUM_UNITS3.domain.values:
                for num_units4 in HP_NUM_UNITS4.domain.values:
#       for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
#                     for optimizer in HP_OPTIMIZER.domain.values:
                    hparams = {
                      HP_NUM_UNITS1: num_units1,
                      HP_NUM_UNITS2: num_units2,
                      HP_NUM_UNITS3: num_units3,
                      HP_NUM_UNITS4: num_units4,
                    #                       HP_DROPOUT: dropout_rate,
        #                       HP_OPTIMIZER: optimizer,
                      HP_BATCHSIZE: batch_size
                    }
                    run_name = "run-%d" % session_num
                    print('--- Starting trial: %s' % run_name)
        #                 print({h.name: hparams[h] for h in hparams})
                    run('logs/hparam_tuning/' + run_name, hparams)
                    session_num += 1

print("Total combinations: {}".format(session_num+1))

--- Starting trial: run-0
1/1 [==============================] - 0s 1ms/step - loss: 380.9155 - mean_squared_error: 380.9155
--- Starting trial: run-1
1/1 [==============================] - 0s 1ms/step - loss: 416.3018 - mean_squared_error: 416.3018
--- Starting trial: run-2
1/1 [==============================] - 0s 1ms/step - loss: 353.9688 - mean_squared_error: 353.9688
--- Starting trial: run-3
1/1 [==============================] - 0s 2ms/step - loss: 312.0697 - mean_squared_error: 312.0697
--- Starting trial: run-4
1/1 [==============================] - 0s 2ms/step - loss: 272.7152 - mean_squared_error: 272.7152
--- Starting trial: run-5
1/1 [==============================] - 0s 1ms/step - loss: 290.3640 - mean_squared_error: 290.3640
--- Starting trial: run-6
1/1 [==============================] - 0s 1ms/step - loss: 374.2433 - mean_squared_error: 374.2433
--- Starting trial: run-7
1/1 [==============================] - 0s 2ms/step - loss: 295.4475 - mean_squared_error: 295.4475


1/1 [==============================] - 0s 2ms/step - loss: 402.9681 - mean_squared_error: 402.9681
--- Starting trial: run-13
1/1 [==============================] - 0s 2ms/step - loss: 268.6082 - mean_squared_error: 268.6082
--- Starting trial: run-14
1/1 [==============================] - 0s 2ms/step - loss: 434.9768 - mean_squared_error: 434.9768
--- Starting trial: run-15
1/1 [==============================] - 0s 2ms/step - loss: 322.8079 - mean_squared_error: 322.8079
--- Starting trial: run-16
1/1 [==============================] - 0s 3ms/step - loss: 306.6107 - mean_squared_error: 306.6107
--- Starting trial: run-17
1/1 [==============================] - 0s 3ms/step - loss: 307.4690 - mean_squared_error: 307.4690
--- Starting trial: run-18
1/1 [==============================] - 0s 2ms/step - loss: 350.3910 - mean_squared_error: 350.3910
--- Starting trial: run-19
1/1 [==============================] - 0s 1ms/step - loss: 481.1599 - mean_squared_error: 481.1599
--- Starting trial:

1/1 [==============================] - 0s 3ms/step - loss: 315.9349 - mean_squared_error: 315.9349
--- Starting trial: run-23
1/1 [==============================] - 0s 3ms/step - loss: 307.5464 - mean_squared_error: 307.5464
--- Starting trial: run-24
1/1 [==============================] - 0s 948us/step - loss: 325.5903 - mean_squared_error: 325.5903
--- Starting trial: run-25
1/1 [==============================] - 0s 2ms/step - loss: 310.3842 - mean_squared_error: 310.3842
--- Starting trial: run-26
1/1 [==============================] - 0s 2ms/step - loss: 265.2167 - mean_squared_error: 265.2167
--- Starting trial: run-27
1/1 [==============================] - 0s 1ms/step - loss: 443.7466 - mean_squared_error: 443.7466
--- Starting trial: run-28
1/1 [==============================] - 0s 2ms/step - loss: 329.4736 - mean_squared_error: 329.4736
--- Starting trial: run-29
1/1 [==============================] - 0s 2ms/step - loss: 325.6570 - mean_squared_error: 325.6570
--- Starting tria

1/1 [==============================] - 0s 2ms/step - loss: 426.6158 - mean_squared_error: 426.6158
--- Starting trial: run-33
1/1 [==============================] - 0s 2ms/step - loss: 323.9776 - mean_squared_error: 323.9776
--- Starting trial: run-34
1/1 [==============================] - 0s 2ms/step - loss: 465.2300 - mean_squared_error: 465.2300
--- Starting trial: run-35
1/1 [==============================] - 0s 1ms/step - loss: 273.6503 - mean_squared_error: 273.6503
--- Starting trial: run-36
1/1 [==============================] - 0s 1ms/step - loss: 440.9406 - mean_squared_error: 440.9406
--- Starting trial: run-37
1/1 [==============================] - 0s 869us/step - loss: 276.9419 - mean_squared_error: 276.9419
--- Starting trial: run-38
1/1 [==============================] - 0s 1ms/step - loss: 410.8710 - mean_squared_error: 410.8710
--- Starting trial: run-39
1/1 [==============================] - 0s 1ms/step - loss: 335.5653 - mean_squared_error: 335.5653
--- Starting tria

1/1 [==============================] - 0s 1ms/step - loss: 306.1477 - mean_squared_error: 306.1477
--- Starting trial: run-43
1/1 [==============================] - 0s 1ms/step - loss: 287.7550 - mean_squared_error: 287.7550
--- Starting trial: run-44
1/1 [==============================] - 0s 1ms/step - loss: 429.5038 - mean_squared_error: 429.5038
--- Starting trial: run-45
1/1 [==============================] - 0s 896us/step - loss: 488.3513 - mean_squared_error: 488.3513
--- Starting trial: run-46
1/1 [==============================] - 0s 1ms/step - loss: 412.7833 - mean_squared_error: 412.7833
--- Starting trial: run-47
1/1 [==============================] - 0s 1ms/step - loss: 295.4512 - mean_squared_error: 295.4512
--- Starting trial: run-48
1/1 [==============================] - 0s 781us/step - loss: 441.9804 - mean_squared_error: 441.9804
--- Starting trial: run-49
1/1 [==============================] - 0s 2ms/step - loss: 381.1492 - mean_squared_error: 381.1492
--- Starting tr

...

1/1 [==============================] - 0s 686us/step - loss: 360.1063 - mean_squared_error: 360.1063
--- Starting trial: run-723
1/1 [==============================] - 0s 666us/step - loss: 341.3438 - mean_squared_error: 341.3438
--- Starting trial: run-724
1/1 [==============================] - 0s 946us/step - loss: 360.1378 - mean_squared_error: 360.1378
--- Starting trial: run-725
1/1 [==============================] - 0s 884us/step - loss: 363.5002 - mean_squared_error: 363.5002
--- Starting trial: run-726
1/1 [==============================] - 0s 698us/step - loss: 344.2397 - mean_squared_error: 344.2397
--- Starting trial: run-727
1/1 [==============================] - 0s 814us/step - loss: 353.8432 - mean_squared_error: 353.8432
--- Starting trial: run-728
1/1 [==============================] - 0s 857us/step - loss: 365.0098 - mean_squared_error: 365.0098
--- Starting trial: run-729
1/1 [==============================] - 0s 963us/step - loss: 353.3065 - mean_squared_error: 353.3

1/1 [==============================] - 0s 961us/step - loss: 358.2620 - mean_squared_error: 358.2620
--- Starting trial: run-733
1/1 [==============================] - 0s 906us/step - loss: 346.8931 - mean_squared_error: 346.8931
--- Starting trial: run-734
1/1 [==============================] - 0s 751us/step - loss: 333.2063 - mean_squared_error: 333.2063
--- Starting trial: run-735
1/1 [==============================] - 0s 828us/step - loss: 366.0220 - mean_squared_error: 366.0220
--- Starting trial: run-736
1/1 [==============================] - 0s 851us/step - loss: 348.9162 - mean_squared_error: 348.9162
--- Starting trial: run-737
1/1 [==============================] - 0s 909us/step - loss: 371.6054 - mean_squared_error: 371.6054
--- Starting trial: run-738
1/1 [==============================] - 0s 1ms/step - loss: 373.2970 - mean_squared_error: 373.2970
--- Starting trial: run-739
1/1 [==============================] - 0s 666us/step - loss: 365.9738 - mean_squared_error: 365.973

1/1 [==============================] - 0s 875us/step - loss: 344.3940 - mean_squared_error: 344.3940
--- Starting trial: run-743
1/1 [==============================] - 0s 939us/step - loss: 386.3188 - mean_squared_error: 386.3188
--- Starting trial: run-744
1/1 [==============================] - 0s 1ms/step - loss: 369.9803 - mean_squared_error: 369.9803
--- Starting trial: run-745
1/1 [==============================] - 0s 1ms/step - loss: 374.7899 - mean_squared_error: 374.7899
--- Starting trial: run-746
1/1 [==============================] - 0s 885us/step - loss: 376.0906 - mean_squared_error: 376.0906
--- Starting trial: run-747
1/1 [==============================] - 0s 676us/step - loss: 340.4772 - mean_squared_error: 340.4772
--- Starting trial: run-748
1/1 [==============================] - 0s 893us/step - loss: 363.4055 - mean_squared_error: 363.4055
--- Starting trial: run-749
1/1 [==============================] - 0s 707us/step - loss: 359.2764 - mean_squared_error: 359.2764


1/1 [==============================] - 0s 1ms/step - loss: 364.2725 - mean_squared_error: 364.2725
--- Starting trial: run-753
1/1 [==============================] - 0s 929us/step - loss: 354.6826 - mean_squared_error: 354.6826
--- Starting trial: run-754
1/1 [==============================] - 0s 927us/step - loss: 362.7003 - mean_squared_error: 362.7003
--- Starting trial: run-755
1/1 [==============================] - 0s 792us/step - loss: 356.4818 - mean_squared_error: 356.4818
--- Starting trial: run-756
1/1 [==============================] - 0s 667us/step - loss: 342.5199 - mean_squared_error: 342.5199
--- Starting trial: run-757
1/1 [==============================] - 0s 836us/step - loss: 362.3622 - mean_squared_error: 362.3622
--- Starting trial: run-758
1/1 [==============================] - 0s 871us/step - loss: 396.7376 - mean_squared_error: 396.7376
--- Starting trial: run-759
1/1 [==============================] - 0s 1ms/step - loss: 352.8404 - mean_squared_error: 352.8404


1/1 [==============================] - 0s 876us/step - loss: 407.1452 - mean_squared_error: 407.1452
--- Starting trial: run-763
1/1 [==============================] - 0s 751us/step - loss: 352.2667 - mean_squared_error: 352.2667
--- Starting trial: run-764
1/1 [==============================] - 0s 725us/step - loss: 357.3218 - mean_squared_error: 357.3218
--- Starting trial: run-765
1/1 [==============================] - 0s 977us/step - loss: 346.0368 - mean_squared_error: 346.0368
--- Starting trial: run-766
1/1 [==============================] - 0s 1ms/step - loss: 354.0978 - mean_squared_error: 354.0978
--- Starting trial: run-767
1/1 [==============================] - 0s 989us/step - loss: 348.1474 - mean_squared_error: 348.1474
Total combinations: 769


In [14]:
!kill 7439
%tensorboard --logdir logs/hparam_tuning

/bin/sh: 1: kill: No such process



Reusing TensorBoard on port 6006 (pid 22630), started 0:21:53 ago. (Use '!kill 22630' to kill it.)